In [27]:
pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install seaborn

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
!pip install pandas
!pip install snowflake-ml-python
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
pip install ydata-profiling --upgrade

Requirement already up-to-date: ydata-profiling in /opt/conda/lib/python3.9/site-packages (4.10.0)
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
import seaborn as sns

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [ ]:
type(sf_df)

In [ ]:
df=sf_df.to_pandas()

In [ ]:
type(df)

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.corr(numeric_only=True)

In [ ]:
from ydata_profiling import ProfileReport
profile=ProfileReport(df,explorative=True)
profile.to_file("autoeda.html")